In [5]:
import cryptofeed

In [7]:
cryptofeed

<module 'cryptofeed' (namespace)>

In [1]:

from cryptofeed.callback import BookCallback, TickerCallback, TradeCallback
from cryptofeed.defines import BID, ASK, FUNDING, L2_BOOK, OPEN_INTEREST, TICKER, TRADES
from cryptofeed.exchanges import Deribit


async def trade(feed, symbol, order_id, timestamp, side, amount, price, receipt_timestamp):
    print(f"Timestamp: {timestamp} Feed: {feed} Pair: {symbol} ID: {order_id} Side: {side} Amount: {amount} Price: {price}")


async def ticker(feed, symbol, bid, ask, timestamp, receipt_timestamp):
    print(f'Feed: {feed} Pair: {symbol} Bid: {bid} Ask: {ask}')


async def book(feed, symbol, book, timestamp, receipt_timestamp):
    print(f'Timestamp: {timestamp} Feed: {feed} Pair: {symbol} Book Bid Size is {len(book[BID])} Ask Size is {len(book[ASK])}')


async def funding(**kwargs):
    print(f'Funding {kwargs}')


async def oi(feed, symbol, open_interest, timestamp, receipt_timestamp):
    print(f'Timestamp: {timestamp} Feed: {feed} Pair: {symbol} open interest: {open_interest}')


def main():
    f = FeedHandler()

    # Deribit can't handle 400+ simultaneous requests, so if all
    # instruments are needed they should be fed in the different calls

    sub = {TRADES: ["BTC-PERPETUAL"], TICKER: ['ETH-PERPETUAL'], FUNDING: ['ETH-PERPETUAL'], OPEN_INTEREST: ['ETH-PERPETUAL']}
    f.add_feed(Deribit(subscription=sub, callbacks={OPEN_INTEREST: oi, FUNDING: funding, TICKER: TickerCallback(ticker), TRADES: TradeCallback(trade)}))
    print(f)
    f.add_feed(Deribit(symbols=['BTC-PERPETUAL'], channels=[L2_BOOK], callbacks={L2_BOOK: BookCallback(book)}))

    f.run()


if __name__ == '__main__':
    main()

ImportError: cannot import name 'FeedHandler' from 'cryptofeed' (unknown location)

In [ ]:
'''
Copyright (C) 2021-2021  Cryptofeed contributors

Please see the LICENSE file for the terms and conditions
associated with this software.
'''
from functools import partial

from cryptofeed import FeedHandler
from cryptofeed.defines import BOOK_DELTA, FUNDING, BITFINEX, L2_BOOK, L3_BOOK, TICKER, TRADES


async def print_all(*args, **kwargs):
    print(args, kwargs)


def main():

    config = {'log': {'filename': 'demo_bitfinex.log', 'level': 'DEBUG'}}
    f = FeedHandler(config=config)

    callbacks = {
        FUNDING: partial(print_all, FUNDING),
        TRADES: partial(print_all, TRADES),
        TICKER: partial(print_all, TICKER),
        L2_BOOK: partial(print_all, L2_BOOK),
        L3_BOOK: partial(print_all, L3_BOOK),
        BOOK_DELTA: partial(print_all, BOOK_DELTA),
    }

    # OK
    f.add_feed(BITFINEX, symbols=['BTC-USD'], channels=[TRADES], callbacks=callbacks)
    f.add_feed(BITFINEX, symbols=['BTC-USD'], channels=[TICKER], callbacks=callbacks)
    f.add_feed(BITFINEX, symbols=['BTC-USD'], channels=[L2_BOOK], callbacks=callbacks)
    f.add_feed(BITFINEX, symbols=['BTC-USD'], channels=[L3_BOOK], callbacks=callbacks)

    # OK: TRADES FUNDING
    f.add_feed(BITFINEX, symbols=['BTC'], channels=[FUNDING], callbacks=callbacks)

    # Bad: following line subscribes to TRADES /!\
    f.add_feed(BITFINEX, symbols=['BTC-USD'], channels=[FUNDING], callbacks=callbacks)

    # Bad: following line subscribes to FUNDING /!\
    f.add_feed(BITFINEX, symbols=['BTC'], channels=[TRADES], callbacks=callbacks)

    # Warning: TICKER FUNDING not yet implemented -> do nothing
    f.add_feed(BITFINEX, symbols=['BTC'], channels=[TICKER], callbacks=callbacks)

    f.run()


if __name__ == '__main__':
    main()


In [11]:
f = FeedHandler()
?f.add_feed